In [18]:
import $ from "https://deno.land/x/dax@0.39.1/mod.ts";

const test = await $`deno lint --json`.stdout("piped").stderr("piped")
  .captureCombined().noThrow();

export interface Diagnostic {
  range: {
    start: { line: number; col: number; bytePos: number };
    end: { line: number; col: number; bytePos: number };
  };
  filename: string;
  message: string;
  code: string;
  hint: string;
}

export interface LintResults {
  diagnostics: Diagnostic[];
}

const jsonOutput: LintResults = JSON.parse(test.combined);


In [17]:
interface Diagnostic {
  range: {
    start: { line: number; col: number; bytePos: number };
    end: { line: number; col: number; bytePos: number };
  };
  filename: string;
  message: string;
  code: string;
  hint: string;
}

interface LintResults {
  diagnostics: Diagnostic[];
}

const groupedByCode = (jsonOutput as LintResults).diagnostics.reduce(
  (acc, diagnostic) => {
    const { code } = diagnostic;
    if (!acc[code]) {
      acc[code] = [];
    }
    acc[code].push(diagnostic);
    return acc;
  },
  {} as Record<string, Diagnostic[]>,
);

for (const [code, errors] of Object.entries(groupedByCode)) {
  console.log(`${code}: ${errors.length}`);
}


no-explicit-any: 65


In [16]:
import { toFileUrl } from "std/path/mod.ts";

interface Diagnostic {
  range: {
    start: { line: number; col: number; bytePos: number };
    end: { line: number; col: number; bytePos: number };
  };
  filename: string;
  message: string;
  code: string;
  hint: string;
}

interface LintResults {
  diagnostics: Diagnostic[];
}

const cwd = new URL("./", toFileUrl(Deno.cwd())).pathname;

const filenamesAndCountsByCode: Record<
  string,
  { files: string[]; count: number }
> = Object.keys(groupedByCode).reduce((acc, code) => {
  const diagnostics = groupedByCode[code];
  const filenames = diagnostics.map((diagnostic) =>
    diagnostic.filename.replace(cwd, "")
  );

  const count = diagnostics.length;

  const uniqueFilenames = [...new Set(filenames)];

  acc[code] = {
    files: uniqueFilenames,
    count: count,
  };

  return acc;
}, {} as Record<string, { files: string[]; count: number }>);

console.log(filenamesAndCountsByCode);


{
  "no-explicit-any": {
    files: [
      "lib/components/blocks/form/fields/array.tsx",
      "lib/components/blocks/form/fields/enum.tsx",
      "lib/components/blocks/form/fields/object.tsx",
      "lib/components/blocks/form/fields/radio-group.tsx",
      "lib/components/blocks/form/types.ts",
      "lib/components/blocks/form/utils.ts",
      "lib/hooks/use-ui.ts",
      "lib/plugins/api/errors.ts",
      "lib/plugins/api/utils.ts"
    ],
    count: 65
  }
}
